In [1]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

# os.environ['JAX_PLATFORM_NAME'] = "cpu"
# os.environ['JAX_PLATFORMS'] = "cpu"

In [2]:
import jax.numpy as jnp
import jax

import numpy as onp

import matplotlib.pyplot as plt
import random

from simulation.simulate_full import run_entire_simulation

from analyzers import defaultvalues as dv, database, loss as loss_anaylzer, gradients as grad_analyzer

database.set_filename("../data/grad_analyzer/nmolecules_scan.npz")

/home/florian/anaconda3/envs/jaxidp/lib/python3.11/site-packages/google/colab/data_table.py:30: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  from IPython.utils import traitlets as _traitlets


In [3]:
N_GRADIENTS = 3
runtime_values = onp.array([12, 20, 28, 36, 44])
nsteps_intervals = runtime_values * 1000 // 2
print(nsteps_intervals)

[ 6000 10000 14000 18000 22000]


In [4]:
def print_info(grads):
    amean, astd = grad_analyzer.analyze_gradients_absolute(grads)
    mmean, mstd = grad_analyzer.analyze_gradients_magnitudal(grads)

    print(f"Absolute mean: {amean}, Absolute std: {astd}")
    print(f"Magnitudal mean: {mmean}, Magnitudal std: {mstd}")

In [ ]:
for nsteps in nsteps_intervals:
    existing_keys = database.get_existing_keys()
    if nsteps in existing_keys:
        print(f"Skipping {nsteps}, was already computed")
        continue

    grads = []
    print(f"==== for {nsteps//500}ps ====")
    for i in range(1, N_GRADIENTS + 1):
        key = random.randrange(0, 20000)
        def simulation_wrapper(LJ_SIGMA_OO: float) -> float:
            prediction = run_entire_simulation(
                LJ_SIGMA_OO, 
                nsteps, 
                dv.N_MOLECULES_PER_AXIS, 
                dv.N_SNAPSHOTS, 
                dv.N_Q, 
                key)
            reference = run_entire_simulation(
                dv.LJ_SIGMA_OO, 
                nsteps, 
                dv.N_MOLECULES_PER_AXIS, 
                dv.N_SNAPSHOTS, 
                dv.N_Q, 
                key)
            return loss_anaylzer.L1_loss(prediction, reference)
        grad_fn = jax.grad(simulation_wrapper)
        grad = grad_fn(3.1)
        print(grad)
        grads.append(grad)

    print_info(grads)
    database.save_intermediate_result(nsteps, grads)

==== for 12ps ====
-4.696104398344362e+42
-2.1754439892558835e+42
-1.8679809919203282e+43
Absolute mean: 8.517119435601177e+42, Absolute std: 7.259414146408287e+42
Magnitudal mean: 42.76021936726795, Magnitudal std: 0.3863321708163014
==== for 20ps ====


In [ ]:
keys, values = database.load_result()
for key, grads in zip(keys, values):
    print(f"==== for {key//500}ps ====")
    print_info(grads)